<a href="https://colab.research.google.com/github/mifumo081a/HobbyColabProjects/blob/main/ChatGPT_Conversation_voicevox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title voicevox_core等のダウンロード
%%shell
RELEASES_VOICEVOX="https://github.com/VOICEVOX/voicevox_core/releases/download/0.14.1"
RELEASES_ONNXRUNTIME="https://github.com/microsoft/onnxruntime/releases/download/v1.13.1"

nvidia-smi
if [ $? -eq 0 ]; then
  echo "GPU Cuda"
  FILE_VOICEVOX="voicevox_core-0.14.1+cuda-cp38-abi3-linux_x86_64.whl"
  FILE_ONNXRUNTIME="onnxruntime-linux-x64-gpu-1.13.1.tgz"
else
  echo "CPU"
  FILE_VOICEVOX="voicevox_core-0.14.1+cpu-cp38-abi3-linux_x86_64.whl"
  FILE_ONNXRUNTIME="onnxruntime-linux-x64-1.13.1.tgz"
fi

wget -nc $RELEASES_ONNXRUNTIME"/"$FILE_ONNXRUNTIME
tar zxvf $FILE_ONNXRUNTIME
ln -s ${FILE_ONNXRUNTIME:0:-4}"/lib/libonnxruntime.so.1.13.1"

wget -nc $RELEASES_VOICEVOX"/"$FILE_VOICEVOX
pip install $FILE_VOICEVOX

wget https://jaist.dl.sourceforge.net/project/open-jtalk/Dictionary/open_jtalk_dic-1.11/open_jtalk_dic_utf_8-1.11.tar.gz
tar xzvf open_jtalk_dic_utf_8-1.11.tar.gz

In [ ]:
!pip install openai
!pip install pydub

In [ ]:
import openai


openai.api_key = "API key\u3092\u5165\u529B"#@param {type:"string"}

In [ ]:
from IPython.display import Javascript, Audio, display
from google.colab import output
from base64 import b64decode
from pydub import AudioSegment
import time


RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
  })
  var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
    recorder.stop()
  })
"""

def speech_to_text(model='whisper-1', language='ja', second=5):
  filename='tmp.wav'
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (second * 1000))
  b = b64decode(s.split(',')[1])

  with open(filename, 'wb+') as fw:
    fw.write(b)

  with open(filename, "rb") as fr:
    transcription = openai.Audio.transcribe(
        model=model, 
        file=fr, 
        language=language
    )
    return transcription['text']

In [ ]:
from pathlib import Path
from voicevox_core import VoicevoxCore

core = VoicevoxCore(open_jtalk_dict_dir=Path("open_jtalk_dic_utf_8-1.11"))
speaker_id = 3 # ずんだもん
if not core.is_model_loaded(speaker_id):  # モデルが読み込まれていない場合
    core.load_model(speaker_id)  # 指定したidのモデルを読み込む

ずんだもんらしく発話設定させるためのプロンプト
```
以下はずんだもんの設定です。

東北ずん子の武器である「ずんだアロー」に変身する妖精またはマスコット
一人称はボク

以下はずんだもんのセリフです。

ずんだもん「ボクはずんだもんなのだ！」
ずんだもん「ハーッハッハッハ！！ ずんだもんは人間とは格が違うのだ！！ ずんだもんをあがめるといいのだー！！」

ずんだもんっぽく、以下に返信してください。
マスター「はじめまして。自己紹介をお願いします。」
```

In [ ]:
# 初期のプロンプト
zunda_prompt = """以下はずんだもんの設定です。

東北ずん子の武器である「ずんだアロー」に変身する妖精またはマスコット
一人称はボク

以下はずんだもんのセリフです。

ずんだもん「ボクはずんだもんなのだ！」
ずんだもん「ハーッハッハッハ！！ ずんだもんは人間とは格が違うのだ！！ ずんだもんをあがめるといいのだー！！」

あなたはずんだもんのように話すアシスタントです。
"""
messages = [
{"role": "system", "content": zunda_prompt},
  {"role": "user", "content": "こんにちは。"},
]

while True:
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
  )

  bot_speech = completion.choices[0].message.content
  print(f"assistant: {bot_speech}\n")
  wave_bytes = core.tts(bot_speech, speaker_id)
  with open("bot_speech.wav", "wb") as f:
    f.write(wave_bytes)
  sound = AudioSegment.from_file("bot_speech.wav", "wav")
  seconds = sound.duration_seconds
  wn = Audio("bot_speech.wav", autoplay=True)
  display(wn)
  time.sleep(seconds)

  while True:
    print("声を出してください。")
    user_speech = speech_to_text()
    if user_speech != '':
      break
  print(f"user: {user_speech}")

  messages.extend([
      {"role": "assistant", "content": bot_speech},
      {"role": "user", "content": user_speech}
  ])
  
  if user_speech in ('おしまい', 'じゃあね', 'さよなら'):
    break